In [49]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import dh, padding, rsa, utils
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import hashlib
import json
parameters = dh.generate_parameters(generator=2, key_size=512, backend=default_backend())

In [50]:
class User: 
    def __init__(self, name): 
        self.name = name
        
        self.identity = parameters.generate_private_key()
        self.identity_public_key = self.identity.public_key()

        self.Ephemeral = parameters.generate_private_key()
        self.Ephemeral_public_key = self.Ephemeral.public_key()

        # Need a way to generate this
        self.Signed_prekey = parameters.generate_private_key()
        self.Signed_prekey_public_key = self.Signed_prekey.public_key()

        self.One_time_prekey = parameters.generate_private_key()
        self.One_time_prekey_public_key = self.One_time_prekey.public_key()
    
        # Need a way to generate this 
        self.Signature = parameters.generate_private_key()
        self.Signature_public_key = self.Signature.public_key()

class server_user: 
    def __init__(self, name, identity, signed_prekey, onetime_prekey, signature):
        self.name = name
        self.identity_public_key = identity
        self.Signed_prekey_public_key = signed_prekey
        self.One_time_prekey_public_key = onetime_prekey
        self.Signature_public_key = signature

class initial_message: 
    def  __init__(self, name, identity, ephemeral):
        self.name = name
        self.identity_public_key = identity
        self.Ephemeral_public_key = ephemeral 

In [51]:
Alice = User("Alice")
Bob = User("Bob")

Server = {}

def pushToServer(user):
    Server[user.name] = server_user(user.name, user.identity_public_key, user.Signed_prekey_public_key, user.One_time_prekey_public_key, user.Signature_public_key)

pushToServer(Alice)
pushToServer(Bob)

def initialMessage(user): 
    return initial_message(user.name, user.identity_public_key, user.Ephemeral_public_key)

# Bob_Server = server_user("Bob", Bob.identity_public_key, Bob.Signed_prekey_public_key, Bob.One_time_prekey_public_key, Bob.Signature_public_key)
# Alice_Server = server_user("Alice", Alice.identity_public_key, Alice.Signed_prekey_public_key, Alice.One_time_prekey_public_key, Alice.Signature_public_key)

In [52]:
def getDerivedKey1(User1, User2): 
    DH1 = User1.identity.exchange(User2.Signed_prekey_public_key)
    DH2 = User1.Ephemeral.exchange(User2.identity_public_key)
    DH3 = User1.Ephemeral.exchange(User2.Signed_prekey_public_key)
    DH4 = User1.Ephemeral.exchange(User2.One_time_prekey_public_key)
    derived_key = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'handshake data',).derive(DH1 + DH2 + DH3 + DH4)
    return derived_key


def getDerivedKey2(User1, User2): 
    DH1 = User2.Signed_prekey.exchange(User1.identity_public_key)
    DH2 = User2.identity.exchange(User1.Ephemeral_public_key)
    DH3 = User2.Signed_prekey.exchange(User1.Ephemeral_public_key)
    DH4 = User2.One_time_prekey.exchange(User1.Ephemeral_public_key)
    derived_key = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'handshake data',).derive(DH1 + DH2 + DH3 + DH4)
    return derived_key

In [53]:
sk = getDerivedKey1(Alice, Server[Bob.name])

In [54]:
sk = getDerivedKey2(initialMessage(Alice), Bob)

# Symmetric Ratchet

In [55]:
class SymmRatchet(object):
    def __init__(self, key):
        self.state = key

    def next(self, inp=b''):
        # turn the ratchet, changing the state and yielding a new key and IV
        output = HKDF(algorithm=hashes.SHA256(), length=80, salt=b'',
                          info=b'', backend= default_backend() 
                         ).derive(self.state + inp) 
        self.state = output[:32]
        outkey, iv = output[32:64], output[64:]
        return outkey, iv
    
class rachet_user_send: 
    def __init__(self, sk):
        self.sk = sk

    def init_ratchets(self):
        # initialise the root chain with the shared key
        self.root_ratchet = SymmRatchet(self.sk)
        # initialise the sending and recving chains
        self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])

    
class rachet_user_recv: 
    def __init__(self, sk):
        self.sk = sk

    def init_ratchets(self):
        # initialise the root chain with the shared key
        self.root_ratchet = SymmRatchet(self.sk)
        # initialise the sending and recving chains
        self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])


In [56]:
import base64

def b64(msg):
    # base64 encoding helper function
    return base64.encodebytes(msg).decode('utf-8').strip()

bob = rachet_user_recv(sk)
alice = rachet_user_send(sk)

bob.init_ratchets()
alice.init_ratchets()


In [60]:
print('[Alice]\tsend ratchet:', list(map(b64, alice.send_ratchet.next())))
print('[Bob]\trecv ratchet:', list(map(b64, bob.recv_ratchet.next())))
print('[Alice]\trecv ratchet:', list(map(b64, alice.recv_ratchet.next())))
print('[Bob]\tsend ratchet:', list(map(b64, bob.send_ratchet.next())))

[Alice]	send ratchet: ['IeiZHq7uXwziS0QNYE+3u6UtmCG7htn82YffCHw0SU8=', 'p7iMej0F2H5iY+rPSJdaDQ==']
[Bob]	recv ratchet: ['IeiZHq7uXwziS0QNYE+3u6UtmCG7htn82YffCHw0SU8=', 'p7iMej0F2H5iY+rPSJdaDQ==']
[Alice]	recv ratchet: ['g0zXDlzQII96yYJGCDFdsHOHWbM/7b6AipBN0ztJln0=', 'Uic/TvXqiAJKz+gIH1qG6Q==']
[Bob]	send ratchet: ['g0zXDlzQII96yYJGCDFdsHOHWbM/7b6AipBN0ztJln0=', 'Uic/TvXqiAJKz+gIH1qG6Q==']


In [61]:
# import AES
from Crypto.Cipher import AES

def pad(msg):
    # pkcs7 padding
    num = 16 - (len(msg) % 16)
    return msg + bytes([num] * num)

def unpad(msg):
    # remove pkcs7 padding
    return msg[:-msg[-1]]

In [62]:
class SymmRatchet(object):
    def __init__(self, key):
        self.state = key

    def next(self, inp=b''):
        # turn the ratchet, changing the state and yielding a new key and IV
        output = HKDF(algorithm=hashes.SHA256(), length=80, salt=b'',
                          info=b'', backend= default_backend() 
                         ).derive(self.state + inp)  # takes in the state and the input
        self.state = output[:32]
        outkey, iv = output[32:64], output[64:]
        return outkey, iv
    
class rachet_user_send: 

    def __init__(self, sk):
        self.sk = sk
        self.DHratchet = None

    def init_ratchets(self):
        # initialise the root chain with the shared key
        self.root_ratchet = SymmRatchet(self.sk)
        # initialise the sending and recving chains
        self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])

    def dh_ratchet(self, bob_public):
    # perform a DH ratchet rotation using Bob's public key
        if self.DHratchet is not None:
            # the first time we don't have a DH ratchet yet
            dh_recv = self.DHratchet.exchange(bob_public)
            shared_recv = self.root_ratchet.next(dh_recv)[0]
            # use Bob's public and our old private key
            # to get a new recv ratchet
            self.recv_ratchet = SymmRatchet(shared_recv)
            print('[Alice]\tRecv ratchet seed:', b64(shared_recv))
        # generate a new key pair and send ratchet
        # our new public key will be sent with the next message to Bob
        self.DHratchet = parameters.generate_private_key()
        dh_send = self.DHratchet.exchange(bob_public)
        shared_send = self.root_ratchet.next(dh_send)[0]
        self.send_ratchet = SymmRatchet(shared_send)
        print('[Alice]\tSend ratchet seed:', b64(shared_send))


    def send(self, bob, msg):
        key, iv = self.send_ratchet.next()
        cipher = AES.new(key, AES.MODE_CBC, iv).encrypt(pad(msg))
        print('[Alice]\tSending ciphertext to Bob:', b64(cipher))
        # send ciphertext and current DH public key
        bob.recv(cipher, self.DHratchet.public_key())

    def recv(self, cipher, bob_public_key):
        # receive Bob's new public key and use it to perform a DH
        self.dh_ratchet(bob_public_key)
        key, iv = self.recv_ratchet.next()
        # decrypt the message using the new recv ratchet
        msg = unpad(AES.new(key, AES.MODE_CBC, iv).decrypt(cipher))
        print('[Alice]\tDecrypted message:', msg)


    
class rachet_user_recv: 
    def __init__(self, sk):
        self.sk = sk
        self.DHratchet = parameters.generate_private_key()

    def init_ratchets(self):
        # initialise the root chain with the shared key
        self.root_ratchet = SymmRatchet(self.sk)
        # initialise the sending and recving chains
        self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])
    
    def dh_ratchet(self, alice_public): 
        dh_recv = self.DHratchet.exchange(alice_public)
        shared_recv = self.root_ratchet.next(dh_recv)[0]
        self.recv_ratchet = SymmRatchet(shared_recv)
        print('[Bob]\tRecv ratchet seed:', b64(shared_recv))
        # generate a new key pair and send ratchet
        # our new public key will be sent with the next message to Alice
        self.DHratchet = parameters.generate_private_key()
        dh_send = self.DHratchet.exchange(alice_public)
        shared_send = self.root_ratchet.next(dh_send)[0]
        self.send_ratchet = SymmRatchet(shared_send)
        print('[Bob]\tSend ratchet seed:', b64(shared_send))

    def send(self, alice, msg):
        key, iv = self.send_ratchet.next()
        cipher = AES.new(key, AES.MODE_CBC, iv).encrypt(pad(msg))
        print('[Bob]\tSending ciphertext to Alice:', b64(cipher))
        # send ciphertext and current DH public key
        alice.recv(cipher, self.DHratchet.public_key())

    def recv(self, cipher, alice_public_key):
        # receive Alice's new public key and use it to perform a DH
        self.dh_ratchet(alice_public_key)
        key, iv = self.recv_ratchet.next()
        # decrypt the message using the new recv ratchet
        msg = unpad(AES.new(key, AES.MODE_CBC, iv).decrypt(cipher))
        print('[Bob]\tDecrypted message:', msg)



# Diffie Hellman Ratchet

In [63]:
Al = rachet_user_send(sk)
Bb = rachet_user_recv(sk)

Al.init_ratchets()
Bb.init_ratchets()

In [64]:
Al.dh_ratchet(Bb.DHratchet.public_key())

[Alice]	Send ratchet seed: e9NhUFAcBnNwRL0sl82FU3mS34LplnmfKdjUEVd+Ogs=
